# Transfer Learning

We will use pre-trained networks to solve challenging problems in computer vision. Specifically, we will use networks trained on [ImageNet](http://www.image-net.org/) [available from torchvision](http://pytorch.org/docs/stable/torchvision/models.html).

ImageNet is a massive dataset with over 1 million labeled images in 1000 categories. It is used to train deep neural networks using an architecture called convolutional layers.

Once trained, these models work astonishingly well as feature detectors for images they were not trained on. Using a pre-trained network on images not in the training set is called transfer learning. Here we will use transfer learning to train a network that can classify our cat and dog photos with near perfect accuracy.

With `torchvision.models` you can download these pre-trained networks and use them in your application. We will include `models` in our imports now.

In [50]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import matplotlib.pyplot as plt

import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
from torchvision import datasets, transforms, models

Most of the pretrained models require the input to be 224x224 images. We also need to match the normalization used when the models were trained. Each color channel was normalized separately, the means are `[0.485, 0.456, 0.406]` and the standard deviations are `[0.229, 0.224, 0.225]`. 

In [64]:
# Uncomment these lines to download and unzip files as  directory
#!wget https://s3.amazonaws.com/content.udacity-data.com/nd089/Cat_Dog_data.zip
#!unzip Cat_Dog_data.zip

In [53]:
data_dir = 'Cat_Dog_data'

# Define Transforms for the training data and testing data
train_transforms = transforms.Compose([transforms.RandomRotation(30),
                                       transforms.RandomResizedCrop(224),
                                       transforms.RandomHorizontalFlip(),
                                       transforms.ToTensor(),
                                       transforms.Normalize([0.485, 0.456, 0.406],
                                                            [0.229, 0.224, 0.225])])

test_transforms = transforms.Compose([transforms.Resize(255),
                                      transforms.CenterCrop(224),
                                      transforms.ToTensor(),
                                      transforms.Normalize([0.485, 0.456, 0.406],
                                                           [0.229, 0.224, 0.225])])

# Pass transforms in here
train_data = datasets.ImageFolder(data_dir + '/train', transform=train_transforms)
test_data = datasets.ImageFolder(data_dir + '/test', transform=test_transforms)

trainloader = torch.utils.data.DataLoader(train_data, batch_size=64, 
                                          shuffle=True)
testloader = torch.utils.data.DataLoader(test_data, batch_size=64)

We can load in a model such as `DenseNet`. Let's print out the model architecture so we can see what is going on.

```python
model = models.densenet121(pretrained=True)
model
```

This model is built out of two main parts, the features and the classifier. The features part is a stack of convolutional layers and overall works as a feature detector that can be fed into a classifier. The classifier part is a single fully-connected layer `(classifier): Linear(in_features=1024, out_features=1000)`. This layer was trained on the ImageNet dataset, so it will not work for our specific problem. That means we need to replace the classifier, but the features will work perfectly on their own. In general, pre-trained networks can be viewed as good feature detectors that can be used as the input for simple feed-forward classifiers.

```python

# Freeze parameters so we don't backprop through them
for param in model.parameters():
  param.requires_grad = False

from collections import OrderedDict

classifier = nn.Sequential(OrderedDict([
                          ('fc1', nn.Linear(1024, 500)),
                          ('relu', nn.ReLU()),
                          ('dropout', nn.Dropout(p=0.2)),
                          ('fc2', nn.Linear(500, 2)),
                          ('output', nn.LogSoftmax(dim=1))
                          ]))

model.classifier = classifier
```

With our model built, we need to train the classifier. However, now we are using a **really deep** neural network. If you try to train this on a CPU like normal, it will take a long, long time. Instead, we are going to use the GPU to do the calculations. The linear algebra computations are done in parallel on the GPU leading to 100x increased training speeds. It is also possible to train on multiple GPUs, further decreasing training time.

PyTorch, along with most other deep learning frameworks, uses [CUDA]() to efficiently compute the forward and backwards passes on the GPU. In PyTorch, you move your model parameters and other tensors to the GPU memory using `model.to('cuda')`. You can move them back from the GPU with `model.to('cpu')` which you will commonly fo when you need to operate on the network output outside of PyTorch. As a demonstration of the increased speed, I will compare how long it takes to perform a forward and backward pass with and without a GPU.

```python
import time

for device in ['cpu', 'cuda']:
  criterion = nn.NLLLoss()

  # Only train the classifier parameters, feature parameters are frozen
  optimizer = optim.Adam(model.classifier.parameters(), lr=0.001)

  model.to(device)

  for ii, (inputs, labels) in enumerate(trainloader):

    # Move input and label tensors to the GPU
    inputs, labels = inputs.to(device), labels.to(device)

    start = time.time()

    outputs = model.forward(inputs)

    loss = criterion(outputs, labels)

    loss.backward()

    optimizer.step()

    if ii==3:
      break
  
  print(f"Device = {device}; Time per batch: {(time.time() - start)/3:.3f} seconds")
```

You can write device agnostic code which will automatically use CUDA if it's enabled like so:

```python
# at beginning of the script
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

...

# then whenever you get a new Tensor or Module
# this won't copy if they are already on the desired device
input = data.to(device)
model = MyModule(...).to(device)
```

From here, you will finish training the model. The process is the same as before except now your model is much more powerful. You should get better than 95% accuracy easily.

>**Exercise:** Train a pretrained model to classify the cat and dog images. Continue with the DenseNet model, or try ResNet which is a good model to try out first. Make sure you are only training the classifier and the parameters for the features part are frozen.

In [54]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [63]:
model = models.resnet50(pretrained=True)
#model

In [56]:
# Turn off gradients for the model
for param in model.parameters():
  param.requires_grad = False

In [57]:
# Define our new classifier
classifier = nn.Sequential(nn.Linear(2048, 512),
                         nn.ReLU(),
                         nn.Dropout(p=0.2),
                         nn.Linear(512, 2),
                         nn.LogSoftmax(dim=1))

model.fc = classifier

In [62]:
#model

In [61]:
criterion = nn.NLLLoss()

optimizer = optim.Adam(model.fc.parameters(), lr=0.003)

model.to(device);

In [60]:
epochs = 1
steps = 0
running_loss = 0
print_every = 5

for epoch in range(epochs):
  for images, labels in trainloader:
    steps += 1

    images, labels = images.to(device), labels.to(device)

    optimizer.zero_grad()

    logps = model(images)

    loss = criterion(logps, labels)

    loss.backward()

    optimizer.step()

    running_loss += loss.item()

    if steps % print_every == 0:
      model.eval()
      test_loss = 0
      accuracy = 0

      for images, labels in testloader:
        images, labels = images.to(device), labels.to(device)
        logps = model(images)
        loss = criterion(logps, labels)
        test_loss += loss.item()

        # Calculate accuracy
        ps = torch.exp(logps)
        top_ps, top_class = ps.topk(1, dim=1)
        equality = top_class == labels.view(*top_class.shape)
        accuracy += torch.mean(equality.type(torch.FloatTensor)).item()

      print(f"Epoch {epoch+1}/{epochs}.."
            f"Train loss: {running_loss/print_every:.3f}.."
            f"Test loss: {test_loss/len(testloader):.3f}.."
            f"Test accuracy: {accuracy/len(testloader):.3f}") 
     
      running_loss = 0
      model.train()

Epoch 1/1..Train loss: 1.526..Test loss: 0.060..Test accuracy: 0.980
Epoch 1/1..Train loss: 0.709..Test loss: 0.207..Test accuracy: 0.934
Epoch 1/1..Train loss: 0.615..Test loss: 0.437..Test accuracy: 0.862
Epoch 1/1..Train loss: 0.512..Test loss: 0.180..Test accuracy: 0.940
Epoch 1/1..Train loss: 0.391..Test loss: 0.077..Test accuracy: 0.969
Epoch 1/1..Train loss: 0.269..Test loss: 0.058..Test accuracy: 0.977
Epoch 1/1..Train loss: 0.194..Test loss: 0.104..Test accuracy: 0.961
Epoch 1/1..Train loss: 0.270..Test loss: 0.078..Test accuracy: 0.970
Epoch 1/1..Train loss: 0.154..Test loss: 0.088..Test accuracy: 0.965
Epoch 1/1..Train loss: 0.244..Test loss: 0.068..Test accuracy: 0.972
Epoch 1/1..Train loss: 0.229..Test loss: 0.056..Test accuracy: 0.978
Epoch 1/1..Train loss: 0.204..Test loss: 0.116..Test accuracy: 0.953
Epoch 1/1..Train loss: 0.214..Test loss: 0.062..Test accuracy: 0.978
Epoch 1/1..Train loss: 0.133..Test loss: 0.063..Test accuracy: 0.977
Epoch 1/1..Train loss: 0.232..Test